In [1]:
from keras.utils import np_utils
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import numpy as np

c:\python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
import pickle
pickle_in = open("cleandata.pickle","rb")
dicta = pickle.load(pickle_in)
X = dicta['x']
Y = dicta['y']
nb_classes = 8
X
X = np.array(X)

In [4]:
X

array([[[  9,   9,   9, ...,  42,  41,  41],
        [  9,   9,   9, ...,  41,  41,  41],
        [  8,   8,   9, ...,  42,  41,  41],
        ...,
        [160, 160, 161, ...,  68,  69,  70],
        [159, 159, 160, ...,  67,  69,  69],
        [159, 159, 160, ...,  67,  69,  69]],

       [[ 10,  10,  10, ...,  92,  93,  93],
        [ 10,  10,  10, ...,  91,  92,  93],
        [ 10,  10,  11, ...,  90,  91,  91],
        ...,
        [ 59,  58,  54, ...,  36,  36,  36],
        [ 61,  60,  56, ...,  38,  39,  39],
        [ 61,  60,  56, ...,  38,  39,  39]],

       [[ 92,  92,  92, ..., 107, 107, 107],
        [ 92,  92,  92, ..., 107, 107, 107],
        [ 92,  92,  92, ..., 106, 106, 106],
        ...,
        [101, 101, 102, ..., 128, 129, 129],
        [101, 101, 102, ..., 129, 130, 130],
        [101, 101, 102, ..., 129, 130, 130]],

       ...,

       [[ 51,  51,  48, ...,  97,  97,  97],
        [ 50,  50,  47, ...,  97,  97,  97],
        [ 45,  45,  42, ...,  97,  97,  97

In [5]:
print(len(X),len(Y))

12758 12758


In [6]:
label=[]
for i in Y:
    var = 0
    if i=="anger":
        var = 1
    elif i=="surprise":
        var = 2
    elif i=="happiness":
        var = 3
    elif i =="neutral":
        var = 0
    elif i=="disgust":
        var = 4
    elif i=="fear":
        var = 5
    elif i=="contempt":
        var = 6
    else: #sadness
        var = 7 
    label.append(var)
Y = np.array(label)
Y

array([0, 3, 0, ..., 0, 3, 0])

In [7]:
y = np_utils.to_categorical(Y, nb_classes).astype(np.float32)
y
# prepare weighting for classes since they're unbalanced
class_totals = y.sum(axis=0)
class_weight = class_totals.max() / class_totals
y

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [8]:
X=X.reshape(12758,350,350,1)
X.shape

(12758, 350, 350, 1)

In [9]:
y.shape

(12758, 8)

Keras Model : 

In [10]:
nb_filters = 32
nb_pool = 2
nb_conv = 3

model = Sequential()

model.add(Conv2D(nb_filters, (nb_conv, nb_conv), activation='relu', input_shape=X.shape[1:]))
model.add(Conv2D(nb_filters, (nb_conv, nb_conv), activation='relu'))
model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 348, 348, 32)      320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 346, 346, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 173, 173, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 173, 173, 32)      0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 957728)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               122589312 
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

In [ ]:
validation_split = 0.10
model.fit(X, y, batch_size=10, class_weight=class_weight, epochs=5, verbose=1, validation_split=validation_split)

Train on 11482 samples, validate on 1276 samples
Epoch 1/5
